In [1]:
import numpy as np
import pandas as pd

In [3]:
data = pd.read_csv('train.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 20 columns):
Id               10000 non-null int64
DistrictId       10000 non-null int64
Rooms            10000 non-null float64
Square           10000 non-null float64
LifeSquare       7887 non-null float64
KitchenSquare    10000 non-null float64
Floor            10000 non-null int64
HouseFloor       10000 non-null float64
HouseYear        10000 non-null int64
Ecology_1        10000 non-null float64
Ecology_2        10000 non-null object
Ecology_3        10000 non-null object
Social_1         10000 non-null int64
Social_2         10000 non-null int64
Social_3         10000 non-null int64
Healthcare_1     5202 non-null float64
Helthcare_2      10000 non-null int64
Shops_1          10000 non-null int64
Shops_2          10000 non-null object
Price            10000 non-null float64
dtypes: float64(8), int64(9), object(3)
memory usage: 1.5+ MB


In [4]:
data = data.drop('Healthcare_1', axis=1)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 19 columns):
Id               10000 non-null int64
DistrictId       10000 non-null int64
Rooms            10000 non-null float64
Square           10000 non-null float64
LifeSquare       7887 non-null float64
KitchenSquare    10000 non-null float64
Floor            10000 non-null int64
HouseFloor       10000 non-null float64
HouseYear        10000 non-null int64
Ecology_1        10000 non-null float64
Ecology_2        10000 non-null object
Ecology_3        10000 non-null object
Social_1         10000 non-null int64
Social_2         10000 non-null int64
Social_3         10000 non-null int64
Helthcare_2      10000 non-null int64
Shops_1          10000 non-null int64
Shops_2          10000 non-null object
Price            10000 non-null float64
dtypes: float64(7), int64(9), object(3)
memory usage: 1.4+ MB


In [5]:
data.describe()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Social_1,Social_2,Social_3,Helthcare_2,Shops_1,Price
count,10000.00000,10000.000000,10000.000000,10000.000000,7887.000000,10000.000000,10000.000000,10000.000000,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,8383.40770,50.400800,1.890500,56.315775,37.199645,6.273300,8.526700,12.609400,3.990166e+03,0.118858,24.687000,5352.157400,8.039200,1.319500,4.231300,214138.857399
std,4859.01902,43.587592,0.839512,21.058732,86.241209,28.560917,5.241148,6.775974,2.005003e+05,0.119025,17.532614,4006.799803,23.831875,1.493601,4.806341,92872.293865
min,0.00000,0.000000,0.000000,1.136859,0.370619,0.000000,1.000000,0.000000,1.910000e+03,0.000000,0.000000,168.000000,0.000000,0.000000,0.000000,59174.778028
25%,4169.50000,20.000000,1.000000,41.774881,22.769832,1.000000,4.000000,9.000000,1.974000e+03,0.017647,6.000000,1564.000000,0.000000,0.000000,1.000000,153872.633942
50%,8394.50000,36.000000,2.000000,52.513310,32.781260,6.000000,7.000000,13.000000,1.977000e+03,0.075424,25.000000,5285.000000,2.000000,1.000000,3.000000,192269.644879
75%,12592.50000,75.000000,2.000000,65.900625,45.128803,9.000000,12.000000,17.000000,2.001000e+03,0.195781,36.000000,7227.000000,5.000000,2.000000,6.000000,249135.462171
max,16798.00000,209.000000,19.000000,641.065193,7480.592129,2014.000000,42.000000,117.000000,2.005201e+07,0.521867,74.000000,19083.000000,141.000000,6.000000,23.000000,633233.466570


In [6]:
data.loc[data['HouseYear'] == 20052011, 'HouseYear'] = 2008
data.loc[data['HouseYear'] == 4968, 'HouseYear'] = 1968

In [7]:
data.loc[data['Rooms'] == 19, 'Rooms'] = 1
data.loc[data['Rooms'] == 10, 'Rooms'] = 2

In [46]:
def prepare_square(df):
    df.loc[df['Square'] > (df['Rooms'] * 40 + 40), 'Square'] = df['Rooms'] * 40 + 40
    df.loc[df['Square'] < (df['Rooms'] * 15), 'Square'] = df['Rooms'] * 15
    df.loc[df['Square'] < 15, 'Square'] = 15
    df.loc[df['LifeSquare'] < 15, 'LifeSquare'] = df['Square']
    df.loc[df['Square'] < df['LifeSquare'], 'LifeSquare'] = df['Square']
    df['LifeSquare'] = df['LifeSquare'].fillna(df['Square'])
    return df

In [47]:
data = prepare_square(data)

In [48]:
def prepare_floor(df):
    df.loc[df['HouseFloor'] == 0, 'HouseFloor'] = np.round(df.loc[df['HouseFloor'] != 0, 'HouseFloor'].mean())
    df.loc[df['Floor'] > df['HouseFloor'], 'Floor'] = df['HouseFloor']
    return df

In [49]:
data = prepare_floor(data)

In [50]:
data.describe()

,Id,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Social_1,Social_2,Social_3,Helthcare_2,Shops_1,Price,mean_price
count,10000.00000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.00000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,8383.40770,1.887100,56.061895,42.138657,6.273300,7.918200,12.95910,1984.86600,0.118858,24.687000,5352.157400,8.039200,1.319500,4.231300,214138.857399,214138.857399
std,4859.01902,0.813892,18.626468,19.742271,28.560917,5.186386,6.44346,18.41187,0.119025,17.532614,4006.799803,23.831875,1.493601,4.806341,92872.293865,74575.999270
min,0.00000,0.000000,15.000000,15.000000,0.000000,1.000000,1.00000,1910.00000,0.000000,0.000000,168.000000,0.000000,0.000000,0.000000,59174.778028,82342.775873
25%,4169.50000,1.000000,41.780320,27.723574,1.000000,4.000000,9.00000,1974.00000,0.017647,6.000000,1564.000000,0.000000,0.000000,1.000000,153872.633942,166014.055413
50%,8394.50000,2.000000,52.509275,38.586674,6.000000,7.000000,13.00000,1977.00000,0.075424,25.000000,5285.000000,2.000000,1.000000,3.000000,192269.644879,198777.278269
75%,12592.50000,2.000000,65.894295,51.158617,9.000000,11.000000,17.00000,2001.00000,0.195781,36.000000,7227.000000,5.000000,2.000000,6.000000,249135.462171,248821.481005
max,16798.00000,6.000000,240.000000,233.949309,2014.000000,42.000000,117.00000,2020.00000,0.521867,74.000000,19083.000000,141.000000,6.000000,23.000000,633233.466570,624156.003381


In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 19 columns):
Id               10000 non-null int64
DistrictId       10000 non-null int64
Rooms            10000 non-null float64
Square           10000 non-null float64
LifeSquare       10000 non-null float64
KitchenSquare    10000 non-null float64
Floor            10000 non-null float64
HouseFloor       10000 non-null float64
HouseYear        10000 non-null int64
Ecology_1        10000 non-null float64
Ecology_2        10000 non-null object
Ecology_3        10000 non-null object
Social_1         10000 non-null int64
Social_2         10000 non-null int64
Social_3         10000 non-null int64
Helthcare_2      10000 non-null int64
Shops_1          10000 non-null int64
Shops_2          10000 non-null object
Price            10000 non-null float64
dtypes: float64(8), int64(8), object(3)
memory usage: 1.4+ MB


In [17]:
data['DistrictId'] = data['DistrictId'].astype(str)
data.dtypes

Id                 int64
DistrictId        object
Rooms            float64
Square           float64
LifeSquare       float64
KitchenSquare    float64
Floor            float64
HouseFloor       float64
HouseYear          int64
Ecology_1        float64
Ecology_2         object
Ecology_3         object
Social_1           int64
Social_2           int64
Social_3           int64
Helthcare_2        int64
Shops_1            int64
Shops_2           object
Price            float64
dtype: object

In [18]:
pd.get_dummies(data).head()

,Id,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Social_1,...,DistrictId_96,DistrictId_97,DistrictId_98,DistrictId_99,Ecology_2_A,Ecology_2_B,Ecology_3_A,Ecology_3_B,Shops_2_A,Shops_2_B
0,14038,2.0,47.981561,29.442751,6.0,7.0,9.0,1969,0.089040,33,...,0,0,0,0,0,1,0,1,0,1
1,15053,3.0,65.683640,40.049543,8.0,7.0,9.0,1978,0.000070,46,...,0,0,0,0,0,1,0,1,0,1
2,4765,2.0,44.947953,29.197612,0.0,8.0,12.0,1968,0.049637,34,...,0,0,0,0,0,1,0,1,0,1
3,5809,2.0,53.352981,52.731512,9.0,8.0,17.0,1977,0.437885,23,...,0,0,0,0,0,1,0,1,0,1
4,10783,1.0,39.649192,23.776169,7.0,11.0,12.0,1976,0.012339,35,...,0,0,0,1,0,1,0,1,0,1


In [19]:
mean_price = data.groupby(['DistrictId', 'Rooms'], as_index=False)[['Price']].mean().rename(columns={'Price': 'mean_price'})
mean_price

,DistrictId,Rooms,mean_price
0,0,1.0,141262.601991
1,0,2.0,197998.146681
2,0,3.0,304170.518823
3,1,1.0,147186.814704
4,1,2.0,198777.278269
5,1,3.0,246262.924161
6,1,4.0,277290.705058
7,10,1.0,170966.418015
8,10,2.0,226197.554021
9,10,3.0,288421.658993


In [20]:
data = pd.merge(data, mean_price, on=['DistrictId', 'Rooms'], how='left')
data.head()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Helthcare_2,Shops_1,Shops_2,Price,mean_price
0,14038,35,2.0,47.981561,29.442751,6.0,7.0,9.0,1969,0.089040,B,B,33,7976,5,0,11,B,184966.930730,213160.076297
1,15053,41,3.0,65.683640,40.049543,8.0,7.0,9.0,1978,0.000070,B,B,46,10309,1,1,16,B,300009.450063,303643.563728
2,4765,53,2.0,44.947953,29.197612,0.0,8.0,12.0,1968,0.049637,B,B,34,7759,0,1,3,B,220925.908524,259938.729121
3,5809,58,2.0,53.352981,52.731512,9.0,8.0,17.0,1977,0.437885,B,B,23,5735,3,0,5,B,175616.227217,172368.882269
4,10783,99,1.0,39.649192,23.776169,7.0,11.0,12.0,1976,0.012339,B,B,35,5776,1,2,4,B,150226.531644,155748.415754


### Test

In [21]:
test = pd.read_csv('test.csv')
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 19 columns):
Id               5000 non-null int64
DistrictId       5000 non-null int64
Rooms            5000 non-null float64
Square           5000 non-null float64
LifeSquare       3959 non-null float64
KitchenSquare    5000 non-null float64
Floor            5000 non-null int64
HouseFloor       5000 non-null float64
HouseYear        5000 non-null int64
Ecology_1        5000 non-null float64
Ecology_2        5000 non-null object
Ecology_3        5000 non-null object
Social_1         5000 non-null int64
Social_2         5000 non-null int64
Social_3         5000 non-null int64
Healthcare_1     2623 non-null float64
Helthcare_2      5000 non-null int64
Shops_1          5000 non-null int64
Shops_2          5000 non-null object
dtypes: float64(7), int64(9), object(3)
memory usage: 742.3+ KB


In [23]:
test = test.drop('Healthcare_1', axis=1)
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 18 columns):
Id               5000 non-null int64
DistrictId       5000 non-null int64
Rooms            5000 non-null float64
Square           5000 non-null float64
LifeSquare       3959 non-null float64
KitchenSquare    5000 non-null float64
Floor            5000 non-null int64
HouseFloor       5000 non-null float64
HouseYear        5000 non-null int64
Ecology_1        5000 non-null float64
Ecology_2        5000 non-null object
Ecology_3        5000 non-null object
Social_1         5000 non-null int64
Social_2         5000 non-null int64
Social_3         5000 non-null int64
Helthcare_2      5000 non-null int64
Shops_1          5000 non-null int64
Shops_2          5000 non-null object
dtypes: float64(6), int64(9), object(3)
memory usage: 703.2+ KB


In [29]:
test.describe()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Social_1,Social_2,Social_3,Helthcare_2,Shops_1
count,5000.000000,5000.000000,5000.000000,5000.000000,3959.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.00000,5000.000000
mean,8412.595400,51.279200,1.907000,56.449500,36.158810,5.976800,8.632000,12.601000,1984.392600,0.119874,24.933800,5406.900000,8.262600,1.31940,4.242800
std,4832.674037,44.179466,0.810976,19.092787,17.825287,9.950018,5.483228,6.789213,18.573149,0.120070,17.532202,4026.614773,23.863762,1.47994,4.777365
min,1.000000,0.000000,0.000000,1.378543,0.333490,0.000000,1.000000,0.000000,1908.000000,0.000000,0.000000,168.000000,0.000000,0.00000,0.000000
25%,4221.750000,21.000000,1.000000,41.906231,23.092026,1.000000,4.000000,9.000000,1973.000000,0.019509,6.000000,1564.000000,0.000000,0.00000,1.000000
50%,8320.500000,37.000000,2.000000,52.921340,32.925087,6.000000,7.000000,12.000000,1977.000000,0.072158,25.000000,5285.000000,2.000000,1.00000,3.000000
75%,12598.250000,77.000000,2.000000,66.285129,45.174091,9.000000,12.000000,17.000000,2000.000000,0.195781,36.000000,7287.000000,5.000000,2.00000,6.000000
max,16795.000000,212.000000,6.000000,223.453689,303.071094,620.000000,78.000000,99.000000,2020.000000,0.521867,74.000000,19083.000000,141.000000,6.00000,23.000000


In [51]:
test.loc[test['Rooms'] == 17, 'Rooms'] = 2

In [52]:
test = prepare_square(test)

In [53]:
test = prepare_floor(test)

In [54]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 18 columns):
Id               5000 non-null int64
DistrictId       5000 non-null int64
Rooms            5000 non-null float64
Square           5000 non-null float64
LifeSquare       5000 non-null float64
KitchenSquare    5000 non-null float64
Floor            5000 non-null float64
HouseFloor       5000 non-null float64
HouseYear        5000 non-null int64
Ecology_1        5000 non-null float64
Ecology_2        5000 non-null object
Ecology_3        5000 non-null object
Social_1         5000 non-null int64
Social_2         5000 non-null int64
Social_3         5000 non-null int64
Helthcare_2      5000 non-null int64
Shops_1          5000 non-null int64
Shops_2          5000 non-null object
dtypes: float64(7), int64(8), object(3)
memory usage: 703.2+ KB


In [55]:
test['DistrictId'] = test['DistrictId'].astype(str)
test.dtypes

Id                 int64
DistrictId        object
Rooms            float64
Square           float64
LifeSquare       float64
KitchenSquare    float64
Floor            float64
HouseFloor       float64
HouseYear          int64
Ecology_1        float64
Ecology_2         object
Ecology_3         object
Social_1           int64
Social_2           int64
Social_3           int64
Helthcare_2        int64
Shops_1            int64
Shops_2           object
dtype: object

In [56]:
pd.get_dummies(test).head()

,Id,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Social_1,...,DistrictId_96,DistrictId_97,DistrictId_98,DistrictId_99,Ecology_2_A,Ecology_2_B,Ecology_3_A,Ecology_3_B,Shops_2_A,Shops_2_B
0,725,2.0,49.882643,33.432782,6.0,6.0,14.0,1972,0.310199,11,...,0,0,0,0,0,1,0,1,0,1
1,15856,2.0,69.263183,69.263183,1.0,1.0,1.0,1977,0.075779,6,...,0,0,0,0,0,1,0,1,0,1
2,5480,1.0,15.000000,15.000000,12.0,2.0,5.0,1909,0.000000,30,...,0,0,0,0,0,1,0,1,0,1
3,15664,2.0,73.046609,51.940842,9.0,22.0,22.0,2007,0.101872,23,...,0,0,0,0,0,1,0,1,0,1
4,14275,1.0,47.527111,43.387569,1.0,17.0,17.0,2017,0.072158,2,...,0,0,0,0,0,1,0,1,1,0


In [59]:
test = pd.merge(test, mean_price, on=['DistrictId', 'Rooms'], how='left')
test.head()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Helthcare_2,Shops_1,Shops_2,mean_price
0,725,58,2.0,49.882643,33.432782,6.0,6.0,14.0,1972,0.310199,B,B,11,2748,1,0,0,B,172368.882269
1,15856,74,2.0,69.263183,69.263183,1.0,1.0,1.0,1977,0.075779,B,B,6,1437,3,0,2,B,236250.931918
2,5480,190,1.0,15.000000,15.000000,12.0,2.0,5.0,1909,0.000000,B,B,30,7538,87,5,5,B,NaN
3,15664,47,2.0,73.046609,51.940842,9.0,22.0,22.0,2007,0.101872,B,B,23,4583,3,3,3,B,206507.966738
4,14275,27,1.0,47.527111,43.387569,1.0,17.0,17.0,2017,0.072158,B,B,2,629,1,0,0,A,122710.743033


In [63]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5000 entries, 0 to 4999
Data columns (total 19 columns):
Id               5000 non-null int64
DistrictId       5000 non-null object
Rooms            5000 non-null float64
Square           5000 non-null float64
LifeSquare       5000 non-null float64
KitchenSquare    5000 non-null float64
Floor            5000 non-null float64
HouseFloor       5000 non-null float64
HouseYear        5000 non-null int64
Ecology_1        5000 non-null float64
Ecology_2        5000 non-null object
Ecology_3        5000 non-null object
Social_1         5000 non-null int64
Social_2         5000 non-null int64
Social_3         5000 non-null int64
Helthcare_2      5000 non-null int64
Shops_1          5000 non-null int64
Shops_2          5000 non-null object
mean_price       5000 non-null float64
dtypes: float64(8), int64(7), object(4)
memory usage: 781.2+ KB


In [64]:
mean_price3 = data['Price'].mean()
test['mean_price'] = test['mean_price'].fillna(mean_price3)
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5000 entries, 0 to 4999
Data columns (total 19 columns):
Id               5000 non-null int64
DistrictId       5000 non-null object
Rooms            5000 non-null float64
Square           5000 non-null float64
LifeSquare       5000 non-null float64
KitchenSquare    5000 non-null float64
Floor            5000 non-null float64
HouseFloor       5000 non-null float64
HouseYear        5000 non-null int64
Ecology_1        5000 non-null float64
Ecology_2        5000 non-null object
Ecology_3        5000 non-null object
Social_1         5000 non-null int64
Social_2         5000 non-null int64
Social_3         5000 non-null int64
Helthcare_2      5000 non-null int64
Shops_1          5000 non-null int64
Shops_2          5000 non-null object
mean_price       5000 non-null float64
dtypes: float64(8), int64(7), object(4)
memory usage: 781.2+ KB


### train_test_split

In [66]:
from sklearn.model_selection import train_test_split

In [67]:
train, valid = train_test_split(data, test_size=0.3, random_state=42)

In [68]:
train.shape, valid.shape

((7000, 20), (3000, 20))

#### model LR

In [69]:
from sklearn.linear_model import LinearRegression

In [70]:
model = LinearRegression()

In [95]:
feats = ['Rooms', 'Square', 'LifeSquare', 'Floor', 'HouseFloor', 'HouseYear', 'mean_price']

In [96]:
model.fit(train.loc[:, feats], train['Price'])

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [97]:
pred_train = model.predict(train.loc[:, feats])
pred_train

array([118422.54542038, 308359.87963927, 176847.75506303, ...,
       255140.03541782, 218933.65999473, 324355.78435735])

In [98]:
pred_valid = model.predict(valid.loc[:, feats])
pred_valid

array([169107.81170794, 286048.37915295, 219354.44348547, ...,
       267934.69849089, 123492.1505056 , 258721.14411929])

In [99]:
pred_train.shape, pred_valid.shape

((7000,), (3000,))

In [100]:
from sklearn.metrics import r2_score

In [101]:
r2_score(train['Price'], pred_train)

0.7205843796690922

In [102]:
r2_score(valid['Price'], pred_valid)

0.6907914196170939

In [103]:
test['Price'] = model.predict(test.loc[:, feats])

In [104]:
test['Price'].shape

(5000,)

In [105]:
test['Price'].head()

0    160053.587864
1    233997.871137
2    151939.330868
3    255654.684010
4    142320.293098
Name: Price, dtype: float64

In [106]:
test.loc[:, ['Id', 'Price']].to_csv('SBarinov_predictions.csv', index=None)